In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
import numpy as np
from statistics import mode

### Metadados

In [2]:
metadados = pd.read_csv("metadados.csv")
metadados.drop('Unnamed: 0', axis=1, inplace=True)
metadados.shape

(1274, 6)

In [3]:
metadados.head()

,Features,Role,Level,Keep,Tipo,Cardinality
0,SK_ID_CURR,id,nominal,False,int32,307511
1,TARGET,target,binary,True,int32,2
2,NAME_CONTRACT_TYPE,input,nominal,True,object,2
3,CODE_GENDER,input,nominal,True,object,3
4,FLAG_OWN_CAR,input,nominal,True,object,2


### Tabela de Modelagem

In [4]:
abt = pd.read_parquet('VARS_BUREAU-20230112T005646Z-001/VARS_BUREAU')
abt.shape

ArrowMemoryError: malloc of size 2993927104 failed

### Amostra

In [ ]:
#split = StratifiedShuffleSplit(test_size=0.5)
#for x, y in split.split(abt, abt['TARGET']):
#    amostra1 = abt.iloc[x]
#    amostra2 = abt.iloc[y]
    
#amostra1.shape, amostra2.shape
    
    

In [ ]:
#print(abt['TARGET'].value_counts()/len(abt['TARGET']))

In [ ]:
#print(amostra1['TARGET'].value_counts()/len(amostra1['TARGET']))

### Separar base pra teste (20%)

In [ ]:
split = StratifiedShuffleSplit(test_size=0.2)
for x, y in split.split(abt, abt['TARGET']):
    df00 = abt.iloc[x]
    df00_unseen = abt.iloc[y]
    
print('Dados pra treinamento:' + str(df00.shape))
print('Dados pra teste:' + str(df00_unseen.shape))

In [ ]:
print(df00['TARGET'].value_counts()/len(df00['TARGET']))

In [ ]:
df00

In [ ]:
# Filtrando Data Frame 
vars_numericas_df = metadados[((metadados.Level  == 'ordinal')|(metadados.Level == 'interval')) & (metadados.Role == 'input')]
# Selecionando nomes das variáveis a partir do data frame filtrado
vars_numericas_df

In [ ]:
# Selecionando nomes das variáveis a partir do data frame filtrado
lista_vars_numericas = list(vars_numericas_df['Features'])
lista_vars_numericas

In [ ]:
# Vamos montar um data frame com as variáveis que foram filtradas (Note que a tabela ja tem indice)
df01 = df00[lista_vars_numericas]
df01_t = df00_unseen[lista_vars_numericas]

df01.head()

#### Tratamento dos valores nulos

In [ ]:
# Primeiro vamos verificar a quantidade de missings em todas variáveis
df01_missing = df01.isnull().sum()
df01_missing_t = df01_t.isnull().sum()
df01_missing

In [ ]:
# Salvando quantidade de linhas da tabela em variável
qt_rows = df00.shape[0]

# Gerando data frame com quantidade de missings por variavel
df_pct_missing = pd.DataFrame(df01_missing,columns=['qt_missing'])
df_pct_missing['Features'] = df_pct_missing.index
df_pct_missing['pc_miss'] = (100*df_pct_missing['qt_missing'].divide(qt_rows)).astype(int)
df_pct_missing['qt_rows'] = qt_rows
df_pct_missing.reset_index(drop = True, inplace = True)

df_pct_missing

In [ ]:
# Aplicar o tratamento dos missings substituindo por média 

imp = SimpleImputer(missing_values=np.nan, strategy='median')

df02 = imp.fit_transform(df01[lista_vars_numericas])

df02_t = imp.fit_transform(df01_t[lista_vars_numericas])

df02 = pd.DataFrame(df02, columns= [lista_vars_numericas])

df02_t = pd.DataFrame(df02_t, columns= [lista_vars_numericas])



In [ ]:
df01_t[lista_vars_numericas]

In [ ]:
print('Verificando se ainda tem missings após tratamento:')
df02.isnull().sum()

In [ ]:
df02

### Normalização

#### Normalização por distribuição

In [ ]:
from sklearn.preprocessing import StandardScaler

# Classe responável pela normalização
scaler = StandardScaler()

# Convertendo todas variáveis para tipo float (necessário para normalização)
df03 = df02.astype(float)
df03_t = df02_t.astype(float)

scaled_features = scaler.fit_transform(df03[lista_vars_numericas])
df04 = pd.DataFrame(scaled_features, columns=lista_vars_numericas)

scaled_features_t = scaler.transform(df03_t[lista_vars_numericas])
df04_t = pd.DataFrame(scaled_features_t, columns=lista_vars_numericas)

df04.head(10)

#### Normalização por amplitude

In [ ]:
#from sklearn.preprocessing import MinMaxScaler

# Classe responável pela normalização
#scaler_amplitude = MinMaxScaler(feature_range=(0, 1))

#scaled_features = scaler_amplitude.fit_transform(df03[lista_vars_numericas])
#df05 = pd.DataFrame(scaled_features, columns=lista_vars_numericas)

#df05.head(10)

### Variáveis categóricas

In [ ]:
# Filtrando Data Frame 
vars_char_df = metadados[(metadados.Level  == 'nominal') & (metadados.Role == 'input')]
# Selecionando nomes das variáveis a partir do data frame filtrado
vars_char_df

In [ ]:
# Selecionando nomes das variáveis a partir do data frame filtrado
lista_vars_char = list(vars_char_df['Features'])
lista_vars_char

#### Criar tabela contendo somente variáveis nominais

In [ ]:
df06 = df00[lista_vars_char].sort_index()
df06_t = df00_unseen[lista_vars_char].sort_index()
df06

#### Varificando valores nulos nas categóricas

In [ ]:
df06_missing = df06.isnull().sum()
df06_missing

In [ ]:
# Salvando quantidade de linhas da tabela em variável
qt_rows = df00.shape[0]

# Gerando data frame com quantidade de missings por variavel
df_pct_missing_cat = pd.DataFrame(df06_missing,columns=['qt_missing'])
df_pct_missing_cat['Features'] = df_pct_missing_cat.index
df_pct_missing_cat['pc_miss'] = (100*df_pct_missing_cat['qt_missing'].divide(qt_rows)).astype(int)
df_pct_missing_cat['qt_rows'] = qt_rows
df_pct_missing_cat.reset_index(drop = True, inplace = True)

df_pct_missing_cat

In [ ]:
df07 = df06.copy()
df07_t = df06_t.copy()
df07

In [ ]:
df07['NAME_TYPE_SUITE'] = df07['NAME_TYPE_SUITE'].fillna(mode(df07['NAME_TYPE_SUITE'].dropna()))
df07['OCCUPATION_TYPE'] = df07['OCCUPATION_TYPE'].fillna(mode(df07['OCCUPATION_TYPE'].dropna()))
df07['FONDKAPREMONT_MODE'] = df07['FONDKAPREMONT_MODE'].fillna(mode(df07['FONDKAPREMONT_MODE'].dropna()))
df07['HOUSETYPE_MODE'] = df07['HOUSETYPE_MODE'].fillna(mode(df07['HOUSETYPE_MODE'].dropna()))
df07['WALLSMATERIAL_MODE'] = df07['WALLSMATERIAL_MODE'].fillna(mode(df07['WALLSMATERIAL_MODE'].dropna()))
df07['EMERGENCYSTATE_MODE'] = df07['EMERGENCYSTATE_MODE'].fillna(mode(df07['EMERGENCYSTATE_MODE'].dropna()))

df07

In [ ]:
#teste

df07_t['NAME_TYPE_SUITE'] = df07_t['NAME_TYPE_SUITE'].fillna(mode(df07_t['NAME_TYPE_SUITE'].dropna()))
df07_t['OCCUPATION_TYPE'] = df07_t['OCCUPATION_TYPE'].fillna(mode(df07_t['OCCUPATION_TYPE'].dropna()))
df07_t['FONDKAPREMONT_MODE'] = df07_t['FONDKAPREMONT_MODE'].fillna(mode(df07_t['FONDKAPREMONT_MODE'].dropna()))
df07_t['HOUSETYPE_MODE'] = df07_t['HOUSETYPE_MODE'].fillna(mode(df07_t['HOUSETYPE_MODE'].dropna()))
df07_t['WALLSMATERIAL_MODE'] = df07_t['WALLSMATERIAL_MODE'].fillna(mode(df07_t['WALLSMATERIAL_MODE'].dropna()))
df07_t['EMERGENCYSTATE_MODE'] = df07_t['EMERGENCYSTATE_MODE'].fillna(mode(df07_t['EMERGENCYSTATE_MODE'].dropna()))

df07_t

In [ ]:
#Verificando se ainda há valores nulos

df07_missing = df07_t.isnull().sum()
df07_missing

#### Criando variáveis Dummies

#### Selecionar somente variáveis com cardinalidade inferior a n (neste caso faremos n <= 10)

In [ ]:
# Filtrando Data Frame 
vars_to_dummif_df = metadados[(metadados.Level  == 'nominal') & (metadados.Role == 'input')& (metadados.Cardinality <= 10)]
# Selecionando nomes das variáveis a partir do data frame filtrado
vars_to_dummif_df

In [ ]:
# Selecionando nomes das variáveis a partir do data frame filtrado
lista_vars_dummif = list(vars_to_dummif_df['Features'])
lista_vars_dummif

In [ ]:
df08 = df07[lista_vars_dummif]
df08_t = df07_t[lista_vars_dummif]
df08.head()

In [ ]:
# gerando variáveis dummies
df09 = pd.get_dummies(df08, 
                      columns=lista_vars_dummif,
                      drop_first=True, 
                      prefix = lista_vars_dummif,
                      prefix_sep='_')


df09_t = pd.get_dummies(df08_t, 
                      columns=lista_vars_dummif,
                      drop_first=True, 
                      prefix = lista_vars_dummif,
                      prefix_sep='_')


df09

#### Tratando as variáveis categóricas de alta cardinalidade

In [ ]:
# Filtrando Data Frame (Retiramos a variável Name)
vars_to_le_df = metadados[(metadados.Level  == 'nominal') & 
                          (metadados.Role == 'input') & 
                          (metadados.Cardinality > 10)]

# Selecionando nomes das variáveis a partir do data frame filtrado
vars_to_le_df

In [ ]:
# Selecionando nomes das variáveis a partir do data frame filtrado
lista_vars_le = list(vars_to_le_df['Features'])
lista_vars_le

In [ ]:
df10 = df07[lista_vars_le]
df10_t = df07_t[lista_vars_le]
df10

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()


le_tkt = le.fit_transform(df10['OCCUPATION_TYPE'])
le_tkt_df = pd.DataFrame(le_tkt, columns=['LE_OCCUPATION_TYPE'])

le_cbn = le.fit_transform(df10['ORGANIZATION_TYPE'])
le_cbn_df = pd.DataFrame(le_cbn, columns=['LE_ORGANIZATION_TYPE'])

df11 = pd.merge(le_cbn_df,le_tkt_df, left_index=True, right_index=True)

le_tkt_t = le.fit_transform(df10_t['OCCUPATION_TYPE'])
le_tkt_t_df = pd.DataFrame(le_tkt_t, columns=['LE_OCCUPATION_TYPE'])

le_cbn_t = le.fit_transform(df10_t['ORGANIZATION_TYPE'])
le_cbn_t_df = pd.DataFrame(le_cbn_t, columns=['LE_ORGANIZATION_TYPE'])

df11_t = pd.merge(le_cbn_t_df,le_tkt_t_df, left_index=True, right_index=True)



df11

## Vamos juntar todas as tabelas tratadas

df00 -- ABT de modelagem que contem o target

df04 --> normalizada por distribuição

df05 --> normal por amplitude

df09 --> criação de dummies para nominais de baixa cardinalidade

df11 --> label encoder (estabelecimento de ordem entre as categorias das nominais de alta cardinalidade)

In [ ]:
#df4_9 = pd.merge(df04,df09, how= 'left', left_index=True, right_index=True)
#df_ABT_expl = pd.merge(df4_9,df11, how='left', left_index=True, right_index=True)
#df_ABT_expl['index'] = df_ABT_expl.index

#ABT_df = pd.merge(df_ABT_expl,df00[['TARGET']], how= 'outer', left_index=True, right_index=True)

#print(df4_9.shape)
#print(df_ABT_expl.shape)

#print(ABT_df.shape)

In [ ]:
# Treino

df04['index'] = df04.index
df09['index'] = df04.index

df4_9 = pd.merge(df04,df09, how = 'left', on = 'index')
df_ABT_expl = pd.merge(df4_9,df11, left_index=True, right_index=True)
df_ABT_expl['index'] = df_ABT_expl.index
df00['index'] = df_ABT_expl.index

var_df00 = ['TARGET', 'index']

ABT_df = pd.merge(df_ABT_expl,df00[var_df00], how = 'left', on = 'index')


print(df4_9.shape)
print(df_ABT_expl.shape)

print(ABT_df.shape)


In [ ]:
ABT_df

In [ ]:
# Teste

df04_t['index'] = df04_t.index
df09_t['index'] = df04_t.index

df4_9_t = pd.merge(df04_t,df09_t, how = 'left', on = 'index')
df_ABT_expl_t = pd.merge(df4_9_t,df11_t, left_index=True, right_index=True)
df_ABT_expl_t['index'] = df_ABT_expl_t.index
df00_unseen['index'] = df_ABT_expl_t.index

var_df00_t = ['TARGET', 'index']

ABT_df_t = pd.merge(df_ABT_expl_t,df00_unseen[var_df00_t], how = 'left', on = 'index')


print(df04_t.shape)
print(df_ABT_expl_t.shape)

print(ABT_df_t.shape)

In [ ]:
ABT_df_t.isnull().sum()

#### Salvando tabela tratada para ser utilizada em Machine Learning

In [ ]:
ABT_df.to_csv('ABT_credit_DataPrep_Train2.csv')

In [ ]:
ABT_df_t.to_csv('ABT_credit_DataPrep_Test.csv')